In [214]:
import pandas as pd 
import numpy as np
from datetime import date, datetime, timedelta
import lxml.html
import json
import requests 
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import json
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
url='http://heblocalescomerciales.com/s/todos/todos///?id_location=&id_zone='
session=requests.Session()
session.verify=False 
r = session.get( url.replace(' ',''), headers={
'Upgrade-Insecure-Requests': '1',
'Connection': 'keep-alive',
'Content-Type': 'application/x-www-form-urlencoded', \
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'} )
session.close()
root = lxml.html.fromstring(r.text)

def extrae_spot(url):
    session=requests.Session()
    session.verify=False 
    r = session.get( url.replace(' ',''), headers={
    'Upgrade-Insecure-Requests': '1',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded', \
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'} )
    session.close()
    root = lxml.html.fromstring(r.text)
    ul_item=root.xpath("////*/ul[@class = 'list-li row']/li/strong/text()")
    ul_itemvalue=root.xpath("////*/ul[@class = 'list-li row']/li/text()")
    ul_df=pd.DataFrame([ul_itemvalue],columns=ul_item)
    datos_col=['Pais','Estado','Ciudad','Zona','Área Construida','Tipo de inmueble','Tipo de negocio']
    datos=[]
    if np.sum(np.isin(ul_item,['País:'])*1)>0: datos=datos+[ul_df['País:'].values.tolist()[0]]
    else: datos=datos+['']
    if np.sum(np.isin(ul_item,['Estado:'])*1)>0: datos=datos+[ul_df['Estado:'].values.tolist()[0]]
    else: datos=datos+['']
    if np.sum(np.isin(ul_item,['Ciudad:'])*1)>0: datos=datos+[ul_df['Ciudad:'].values.tolist()[0]]
    else: datos=datos+['']
    if np.sum(np.isin(ul_item,['Zona:'])*1)>0: datos=datos+[ul_df['Zona:'].values.tolist()[0]]
    else: datos=datos+['']
    if np.sum(np.isin(ul_item,['Área Construida:'])*1)>0: datos=datos+[ul_df['Área Construida:'].values.tolist()[0]]
    else: datos=datos+['']
    if np.sum(np.isin(ul_item,['Tipo de inmueble:'])*1)>0: datos=datos+[ul_df['Tipo de inmueble:'].values.tolist()[0]]
    else: datos=datos+['']
    if np.sum(np.isin(ul_item,['Tipo de negocio:'])*1)>0: datos=datos+[ul_df['Tipo de negocio:'].values.tolist()[0]]
    else: datos=datos+['']
    valor=root.xpath("////*/span[@class = 'pr1']/text()")[0].replace('$','').replace(',','')
    periodo=root.xpath("////*/span[@class = 'pr1']/span/text()")[0].replace(' ','')
    moneda=root.xpath("////*/div[@class = 'blq_precio']/text()")[1].replace(' ','').replace("\n",'')
    imagen=root.xpath("////*/div[@class = 'fotorama']/a/@href")
    if len(imagen)>0:
        imagen=imagen[0]
    else:
        imagen=''    
    caracteristicas=str(root.xpath("////*/ul[@class = 'row']/li/text()"))
    columnas=datos_col+['valor']+['periodo']+['moneda']+['imagen']+['caracteristicas']
    return pd.DataFrame([datos+[valor]+[periodo]+[moneda]+[imagen]+[caracteristicas]],columns=columnas)
spots=[]
paginas=root.xpath("////*/a[@class = 'page-link']/@href")
p_actual=paginas[0]
p_siguiente=paginas[len(paginas)-1]
parada=0
curr_spot=[]
while p_actual!=p_siguiente and parada==0 :
    session=requests.Session()
    session.verify=False 
    r = session.get( p_siguiente.replace(' ',''), headers={
    'Upgrade-Insecure-Requests': '1',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded', \
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
         (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'} )
    session.close()
    root = lxml.html.fromstring(r.text)
    lista=root.xpath("////*/div[@class = 'item']")
    curr_spot_antes=curr_spot
    a=lista[0]
    curr_spot=a.xpath("////*/a[@class = 'fill']/@href")
    spots=spots+curr_spot
    if curr_spot==curr_spot_antes:
        parada=1
    paginas=root.xpath("////*/a[@class = 'page-link']/@href")
    p_actual=paginas[0]
    p_siguiente=paginas[len(paginas)-1]

In [217]:
from IPython.display import clear_output
DATA=extrae_spot(spots[0])
cols=DATA.columns.tolist()
n=1
for spot in spots:
    print(n)
    n=n+1
    DATA=pd.concat([DATA,extrae_spot(spot)], axis=0,ignore_index=True)
    clear_output(wait=True)

110


In [220]:
DATA.iloc[1:len(DATA)-1].to_csv('heb.csv',sep=',')

In [222]:
pd.DataFrame(spots).to_csv('heb_links.csv')